In [1]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import tweepy as tw
from datetime import datetime,timedelta
import csv
import pandas as pd
#from api_keys import consumer_key
#from api_keys import consumer_secret
#from api_keys import access_token
#from api_keys import access_secret

In [3]:
## Import API key
consumer_key = "5LEPRwBHiS79rhHimqz1vCbks"
consumer_secret = "61WNxMM5whpzse34PzDnGUBDnit9GWSrvRPzhMFpJHFKUtUKcs"
access_token = "883171617051955205-PElNmMWv3pl0pKMUEtLLZBq7jvBO6rc"
access_secret = "rWMA8IbYoWjFYbhVFi2CURyVCTZGCPatFRt1AzDvk5zR0"

In [4]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tw.API(auth)

**Getting tweets**

In [5]:
# Define function to get the tweets

def get_tweets(search_words, date_since, language, number_of_items):
    
    tweets = tw.Cursor(api.search,
              q=search_words,
              lang=language,
              since=date_since
              ).items(number_of_items)
    
    tweet_list = []
    for tweet in tweets:
      tweet_list.append([tweet.created_at, tweet.text])

# Create dataframe
    df = pd.DataFrame(tweet_list, columns=['Date', 'Tweet'])
    return df

In [6]:
# Call functions to create dataframes
df1 = get_tweets(['Paramount'], '2021-08-16', 'es', 100)
df2 = get_tweets(['Disney Plus'], '2021-08-16', 'es', 100)
df3 = get_tweets(['Amazon Prime Video'], '2021-08-16', 'es', 100)
df4 = get_tweets(['HBO Max'], '2021-08-16', 'es', 100)
df5 = get_tweets(['Start'], '2021-08-16', 'es', 100)
df6 = get_tweets(['Apple TV'], '2021-08-16', 'es', 100)
df7 = get_tweets(['Netflix'], '2021-08-16', 'es', 100)

In [7]:
# Concatenate dataframe
frames = [df1, df2, df3, df4, df5, df6, df7]
result = pd.concat(frames)

In [8]:
# Dataframe
result

,Date,Tweet
0,2021-08-21 05:37:27,"RT @De10mx: #TarjetaDe10 🎥😮🤯 ""Psicosis"" es con..."
1,2021-08-21 05:31:53,"Pagando Netflix, Disney+, Prime Video, HBO max..."
2,2021-08-21 05:25:04,RT @elespectador: Llega un thriller político-s...
3,2021-08-21 05:15:50,"Y ya que hablo de HBO,ocurre algo curioso 🤭\nA..."
4,2021-08-21 05:01:27,@RCachanosky Jajaja! Olivos Coproducción Para...
...,...,...
95,2021-08-21 05:37:33,no jodan esta en Netflix? wtf apenas me entero
96,2021-08-21 05:37:28,@CLaurac23 @RatingColombia @NoticiasCaracol En...
97,2021-08-21 05:37:26,Van a subir Hotel Del Luna a Netflix.\n\nQue v...
98,2021-08-21 05:37:23,@was_charro Porqué no ver Netflix o Amazon? Es...


In [9]:
# Export to cvs sep without commas
result.to_csv('tweets.csv', sep='|', index=False )

**Previous steps**

In [ ]:
#search_words = ['Netflix OR Paramount OR Disney Plus OR HBO Max OR Start OR Apple TV OR Amazon Prime Video']
#search_words = ['Netflix']
#date_since = "2021-08-16"

In [ ]:
#tweets = tw.Cursor(api.search,
#              q=search_words,
#              lang="es",
#              since=date_since,
#              wait_on_rate_limit=True
#              ).items(100)

In [ ]:
#tweet_list = []
#with open('output2.csv', 'w') as f:
#  writer = csv.writer(f, delimiter='|')
#  for tweet in tweets:
#    writer.writerow([tweet.created_at, tweet.text])
#    tweet_list.append([tweet.created_at, tweet.text])

In [ ]:
#tweet_list = []
#for tweet in tweets:
#  tweet_list.append([tweet.created_at, tweet.text])

TweepError: ignored

In [ ]:
#df = pd.DataFrame(tweet_list, columns=['Date', 'Tweet'])

In [ ]:
#df.head

In [ ]:
#search_words = ['Paramount']
#date_since = "2021-08-16"

In [ ]:
#tweets = tw.Cursor(api.search,
#              q=search_words,
#              lang="es",
#              since=date_since
#              ).items(1000)

In [ ]:
#tweet_list = []
#for tweet in tweets:
#  tweet_list.append([tweet.created_at, tweet.text])

In [ ]:
#df2 = pd.DataFrame(tweet_list, columns=['Date', 'Tweet'])

In [ ]:
#frames = [df, df2]
#result = pd.concat(frames)

In [ ]:
#result

,Date,Tweet
0,2021-08-21 04:33:06,☀️ Zenimation de Disney+ convierte escenas clá...
1,2021-08-21 04:25:01,Soy yo o la plataforma de disney plus está bie...
2,2021-08-21 04:17:14,"Amix, préstenme su cuenta de Disney plus y la ..."
3,2021-08-21 04:16:30,Una no puede ver una película online porque te...
4,2021-08-21 04:13:56,@jungkookanojo omgg yo ahora veo star vs las f...
...,...,...
995,2021-08-18 12:34:28,Que las marcas que hayan elegido para represen...
996,2021-08-18 12:34:25,"@JorgeRieraLopez No parece. Vendrá todo aquí, ..."
997,2021-08-18 12:33:51,"RT @enekoruizj: En 2022, tenemos nueva platafo..."
998,2021-08-18 12:33:37,El servicio de streaming SkyShowtime llegará a...
